In [10]:
IMG_SIZE = 512
BATCH_SIZE = 16
EPOCHS = 5
VAL_SPLIT = 0.2

# Data Loading

In [11]:
import tensorflow as tf

dataset_path = "../../data-collection/image-backend/saved_images"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    class_names=['hazard', 'non-hazard'],
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=VAL_SPLIT,
    subset="training",
)

Found 7 files belonging to 2 classes.
Using 6 files for training.


In [12]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    class_names=['hazard', 'non-hazard'],  # Specify the classes to include
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=VAL_SPLIT,
    subset="validation",
)

Found 7 files belonging to 2 classes.
Using 1 files for validation.


In [13]:
train_dataset.class_names

['hazard', 'non-hazard']

# Model Fine-tuning

In [14]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB6
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input


def build_model():
    base_model = EfficientNetB6(
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights="imagenet"
    )
    
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    x = preprocess_input(inputs)

    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [15]:
model = build_model()
model.summary()

Model: "EfficientNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb6 (Functional)     │ (None, 16, 16, 2304)   │    40,960,143 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2304)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pred (Dense)                    │ (None, 1)              │         2,305 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,962,448 (156.26 MB)

 Trainable params: 2,305 (9.00 KB)

 Non-trainable params: 40,960,143 (156.25 MB)

In [16]:
model.fit(train_dataset, epochs=EPOCHS, verbose=2, validation_data=validation_dataset)

Epoch 1/5
1/1 - 26s - 26s/step - accuracy: 0.3333 - loss: 0.7105 - val_accuracy: 1.0000 - val_loss: 0.4106
Epoch 2/5
1/1 - 3s - 3s/step - accuracy: 1.0000 - loss: 0.2877 - val_accuracy: 1.0000 - val_loss: 0.3227
Epoch 3/5
1/1 - 3s - 3s/step - accuracy: 1.0000 - loss: 0.1794 - val_accuracy: 1.0000 - val_loss: 0.3236
Epoch 4/5
1/1 - 3s - 3s/step - accuracy: 1.0000 - loss: 0.0601 - val_accuracy: 1.0000 - val_loss: 0.3175
Epoch 5/5
1/1 - 3s - 3s/step - accuracy: 1.0000 - loss: 0.0340 - val_accuracy: 1.0000 - val_loss: 0.3061


In [17]:
model.save("../models/EfficientNET.keras")